In [1]:
import os
import pandas as pd
from fractions import Fraction
from pandas_parallel_apply import DataFrameParallel, SeriesParallel
from typing import List, Tuple, Dict
from music21 import converter, meter, note, metadata, instrument
import warnings
warnings.filterwarnings('ignore')

In [2]:
DATA_PATH = os.path.join('data', 'MidiCaps')

In [3]:
df = pd.read_json('data/MidiCaps/train.json', lines=True)

In [4]:
df.sample(5)

,location,caption,genre,genre_prob,mood,mood_prob,key,time_signature,tempo,tempo_word,duration,duration_word,chord_summary,chord_summary_occurence,instrument_summary,instrument_numbers_sorted,all_chords,all_chords_timestamps,test_set
10496,lmd_full/1/161743e456a48216fa2f7ef4d61166a8.mid,An energetic and uplifting rock song featuring...,"[rock, pop]","[0.6582, 0.299]","[energetic, happy, positive, motivational, mel...","[0.3244, 0.2215, 0.1265, 0.12140000000000001, ...",G# minor,4/4,140.0,Fast,209,Song,"[Bm, Abm]",15,"[Overdriven Guitar, Drums, Electric Bass, Clav...","[29, 128, 33, 7]","[Abm, Bm, Abm7, Bm, Abm, Bm, Abm, Bm, Abm, Bm,...","[0.557278911, 5.294149659, 5.758548752, 13.839...",False
48646,lmd_full/f/f0648e3578d06c2db13649d6ece5f44f.mid,A melodic electronic song with a touch of ambi...,"[electronic, ambient]","[0.5032, 0.19290000000000002]","[melodic, slow, space, dream, dark]","[0.1458, 0.0925, 0.0698, 0.0463, 0.0453]",Bb minor,4/4,151.0,Fast,209,Song,"[Bbm, Ab, F#, F]",14,"[Synth Strings, Trumpet, Electric Guitar, Synt...","[51, 56, 26, 88, 24, 32, 128]","[Bbm, Ab, F#, F, Bbm, Ab, F#, F, Bbm, Ab, F#, ...","[0.464399092, 3.157913832, 6.315827664, 9.4737...",False
61636,lmd_full/8/8a4a66e11103ac6d6eacd5d79700797b.mid,This energetic electronic dance track in Ab ma...,"[electronic, dance]","[0.6326, 0.2487]","[melodic, space, energetic, dream, retro]","[0.2411, 0.1728, 0.16060000000000002, 0.100700...",Ab major,4/4,133.0,Fast,165,Song,"[Fm, Bbm, C#, Eb]",7,"[Synth Strings, Choir Aahs, Drums, Synth Bass,...","[50, 52, 128, 38, 81, 80, 73, 48]","[Fm, Bbm, C#, Eb, Fm, Bbm, C#, Eb, F7, Eb6, F7...","[0.464399092, 5.387029478, 7.24462585, 9.10222...",False
2978,lmd_full/1/1e0fcd7108b12dce8f03574d339f14a5.mid,"An electronic song with a moderate tempo, feat...","[electronic, rock]","[0.3705, 0.18230000000000002]","[melodic, dark, happy, film, energetic]","[0.12560000000000002, 0.0791, 0.0779, 0.0777, ...",A major,4/4,86.0,Moderate tempo,252,Song,"[E, D6]",18,"[Synth Effects, Rock Organ, Brass Section, Dru...","[103, 18, 61, 128, 30, 48, 48, 38, 91, 33]","[Bm, E, D6, E, Bm, E7, Bm, E, Em7, D6, E, D6, ...","[0.464399092, 2.786394557, 7.337505668, 8.3591...",False
95410,lmd_full/b/ba79db2e033603329fbdcacf9e85fb39.mid,"This lengthy pop piece, infused with electroni...","[pop, electronic]","[0.2214, 0.1658]","[christmas, relaxing, melodic, meditative, happy]","[0.11720000000000001, 0.0926, 0.092, 0.0905000...",E minor,4/4,86.0,Andante,310,Long piece,"[Em, F#m7b5/E, A6, Em7]",2,"[Piano, Electric Guitar, String Ensemble, Acou...","[0, 26, 49, 32, 50, 22, 128, 60, 73]","[E, Em7, A/E, Em7, A, Em7, A7/C#, E, Em7, B7, ...","[0.464399092, 2.972154195, 3.90095238, 7.24462...",False


In [9]:
def filter_by_instrument_count(df, count):
    """
    Filter the dataframe by the number of instruments in the 'instrument_summary' column.

    Args:
        df (pd.DataFrame): The input dataframe.
        count (int): The desired number of instruments in the 'instrument_summary' list.

    Returns:
        pd.DataFrame: A filtered dataframe with rows where the number of instruments matches the given count.
    """
    return df[df['instrument_numbers_sorted'].apply(lambda x: len(x) <= count)]

def filter_by_genre(df, genre):
    """
    Filter the dataframe by the 'genre' column.

    Args:
        df (pd.DataFrame): The input dataframe.
        genre (str): The desired genre to filter by.

    Returns:
        pd.DataFrame: A filtered dataframe with rows where the genre matches the given genre.
    """
    return df[df['genre'].apply(lambda x: genre in x)]


def filter_by_time_signature(df):
    """
    Filter the dataframe by the 'time_signature' column.

    Args:
        df (pd.DataFrame): The input dataframe.
        time_signature (str): The desired time signature to filter by.

    Returns:
        pd.DataFrame: A filtered dataframe with rows where the time signature matches the given time signature.
    """
    return df[df['time_signature'].apply(lambda x: len(x) > 3)]

In [10]:
df3 = filter_by_time_signature(df)

In [14]:
df3.head()

,location,caption,genre,genre_prob,mood,mood_prob,key,time_signature,tempo,tempo_word,duration,duration_word,chord_summary,chord_summary_occurence,instrument_summary,instrument_numbers_sorted,all_chords,all_chords_timestamps,test_set
450,lmd_full/1/11c60c45596d04a97c512d1ac376d080.mid,A melodic pop song with a touch of electronic ...,"[pop, electronic]","[0.2328, 0.1854]","[film, melodic, energetic, dark, happy]","[0.11670000000000001, 0.09870000000000001, 0.0...",F# major,12/8,85.0,Moderate tempo,193,Song,"[B, F#, B, F#, C#]",2,[Piano],"[1, 1]","[F#, B, F#maj7, F#, Abm7, F#maj7, C#, F#, B, B...","[0.464399092, 4.458231292, 6.59446712, 8.35918...",True
504,lmd_full/1/1cc68781bc21d58f5bca4dc6702261f2.mid,A classical Christmas song with a cinematic fe...,"[classical, soundtrack]","[0.4082, 0.2369]","[christmas, meditative, relaxing, film, melodic]","[0.1892, 0.1252, 0.10890000000000001, 0.0791, ...",G major,12/8,95.0,Moderato,154,Song,"[G, C]",3,"[Acoustic Guitar, Electric Bass, Piano, Flute]","[25, 24, 33, 5, 73, 73]","[D6, G, Em7, D, Gmaj7, Em, D, Bm, E7, Cmaj7, G...","[0.464399092, 3.622312925, 5.572789115, 9.3808...",False
725,lmd_full/1/18bf39a1bbf0bd8c1df92a6720a72945.mid,A joyful and relaxing electronic soundtrack fe...,"[electronic, soundtrack]","[0.2255, 0.1945]","[happy, film, relaxing, corporate, background]","[0.1236, 0.08710000000000001, 0.0833, 0.065, 0...",E major,140/8,126.0,Fast,69,Short song,"[Amaj7, Ebm7b5, E, F#7, Em7]",1,[Celesta],[8],"[Amaj7, Ebm7b5, E, F#7, Em7, B, C#7/E#, D6, Ab...","[0.464399092, 4.458231292, 5.665668934, 7.0588...",False
748,lmd_full/1/19d2ea4d8c33cbcf0f1f472849b6dfe7.mid,A slow-paced electronic song with experimental...,"[electronic, experimental]","[0.4066, 0.24150000000000002]","[slow, dark, melodic, happy, love]","[0.20800000000000002, 0.1903, 0.09530000000000...",G major,12/8,93.0,Moderate tempo,267,Song,"[D, G]",16,"[Drums, Synth Lead, Acoustic Guitar, Synth Bass]","[128, 80, 80, 25, 38]","[D, G, D, G, D, G, D, G, D, G, D, G, Em, A, Em...","[0.464399092, 3.993832199, 7.89478458, 11.7028...",True
769,lmd_full/1/1acf91afb1e5d4c3ca352b42c41e9675.mid,A slow and dark classical piece with electroni...,"[classical, electronic]","[0.3163, 0.22540000000000002]","[dark, film, melodic, energetic, epic]","[0.117, 0.10880000000000001, 0.0946, 0.0842000...",A major,12/8,80.0,Slow,100,Short song,"[F#m, Bm, Abm7b5, A, Emaj7]",1,[Piano],"[0, 0]","[F#m, Bm, Abm7b5, A, Emaj7, F#m, B7, E, E7, A6...","[0.464399092, 2.136235827, 4.3653514730000005,...",False


In [5]:
df2 = filter_by_instrument_count(df, 5)
# df2 = filter_by_genre(df2, 'jazz')

In [7]:
df.shape

(168385, 19)

In [6]:
df2.shape

(61667, 19)

In [13]:
df2.shape[0] / df.shape[0] * 100

24.873949579831933

In [35]:
print(df2.iloc[0].to_dict())

{'location': 'lmd_full/1/17655598958db48a34cd882f81402568.mid', 'caption': 'A short electronic ambient song featuring a piano, set in E major with a 4/4 time signature and a Presto tempo. The composition evokes a cinematic, energetic, and melodic atmosphere with hints of epic and dark undertones. The chord progression of B, Emaj7, C#m7, B7, and A adds to its captivating ambiance.', 'genre': ['electronic', 'ambient'], 'genre_prob': [0.30310000000000004, 0.2369], 'mood': ['film', 'energetic', 'melodic', 'epic', 'dark'], 'mood_prob': [0.11620000000000001, 0.11080000000000001, 0.105, 0.0863, 0.0824], 'key': 'E major', 'time_signature': '4/4', 'tempo': 170.0, 'tempo_word': 'Presto', 'duration': 99, 'duration_word': 'Short song', 'chord_summary': ['B', 'Emaj7', 'C#m7', 'B7', 'A'], 'chord_summary_occurence': 5, 'instrument_summary': ['Piano'], 'instrument_numbers_sorted': [0], 'all_chords': ['B', 'Emaj7', 'C#m7', 'B7', 'A', 'B', 'Emaj7', 'C#m7', 'B7', 'A', 'B', 'Emaj7', 'C#m7', 'Emaj7', 'D6',

### TODO: 
* Filter all song with only 2 track
* Filter all song with only 1 or 2 instruments
* Align chords with relative timing by beats

In [8]:
allowed_fractions = [
    # Common binary divisions
    Fraction(1, 1), Fraction(1, 2), Fraction(1, 4),
    Fraction(1, 8), Fraction(1, 16), Fraction(1, 32),

    # Triplets
    Fraction(1, 3), Fraction(2, 3),
    Fraction(1, 6), Fraction(5, 6),
    Fraction(1, 12), Fraction(2, 12), Fraction(3, 12), Fraction(4, 12),  # granular triplets

    # Dotted values (useful for expressive swing/late hits)
    Fraction(3, 4),  # dotted half note
    Fraction(3, 8),  # dotted quarter
    Fraction(3, 16), # dotted eighth
    Fraction(3, 32), # dotted sixteenth

    # Very short triplet pulses (optional but helps with grace notes/swing feel)
    Fraction(1, 24), Fraction(1, 48)
]

def quantize_to_nearest_fraction(value):
    return min(allowed_fractions, key=lambda x: abs(x - value))

def apply_swing_correction(genres, onset, duration):
    swing_genres = {"jazz", "blues", "swing", "boogie-woogie"}
    is_swing = any(tag.lower() in swing_genres for tag in genres)

    if is_swing and duration == Fraction(1, 8):
        position_in_beat = onset % 1
        eighth_index = round(position_in_beat * 8)
        is_first_eighth = eighth_index % 2 == 0
        new_duration = Fraction(2, 3) if is_first_eighth else Fraction(1, 3)
        swing_onset = onset if is_first_eighth else onset - Fraction(1, 8) + Fraction(2, 3)
        return swing_onset, new_duration

    return onset, duration

def quantize_chords_with_time_signatures(
    timestamps: List[float],
    chords: List[str],
    bpm: float,
    time_signature: str,
    genres: List[str] = [],
) -> List[Dict]:
    """
    timestamps: list of chord start times (in seconds)
    chords: list of chord labels (same length as timestamps)
    bpm: fixed tempo in BPM
    """
    assert len(timestamps) == len(chords), "Timestamps and chords must match in length."
    
    seconds_per_beat = 60.0 / bpm
    beats_per_measure = int(time_signature.split('/')[0])

    quantized = []
    for i in range(len(chords)):
        chord = chords[i]
        start_time = timestamps[i]
        end_time = timestamps[i + 1] if i + 1 < len(chords) else start_time + seconds_per_beat * 2
        duration_sec = end_time - start_time

        # Calculate beat onset and duration in beats
        # Use Fraction to avoid floating point precision issues
        beat_onset = Fraction(start_time / seconds_per_beat).limit_denominator(64)
        duration_beats = Fraction(duration_sec / seconds_per_beat).limit_denominator(64)
        # Quantize beat onset and duration to the nearest allowed fraction
        onset_q = quantize_to_nearest_fraction(beat_onset % beats_per_measure, allowed_fractions)
        duration_q = quantize_to_nearest_fraction(duration_beats, allowed_fractions)

        # Apply swing feel (optional)
        # onset_q, duration_q = apply_swing_correction(genres, onset_q, duration_q)

        # Detect grace note
        # is_grace = duration_q < Fraction(1, 32)

        measure = int(beat_onset) // beats_per_measure + 1

        quantized.append({
            "measure": measure,
            "chord": chord,
            "onset": f"{onset_q.numerator}/{onset_q.denominator}",
            "duration": f"{duration_q.numerator}/{duration_q.denominator}"
        })

    # Group by measure
    grouped = {}
    for entry in quantized:
        key = (entry.pop("measure"), entry.pop("time_signature"))
        grouped.setdefault(key, []).append(entry)

    return [
        {"measure": m, "time_signature": ts, "chords": grouped[(m, ts)]}
        for (m, ts) in sorted(grouped)
    ]

In [9]:
def extract_midi_metadata(midi_file):
    """
    Extract metadata from a MIDI file, including the number of tracks, time signatures, tempo, and key signature.

    Args:
        midi_file (str): Path to the MIDI file.

    Returns:
        dict: A dictionary containing metadata such as number of tracks, time signatures, tempo, and key signature.
    """
    # Parse the MIDI file
    score = converter.parse(os.path.join(DATA_PATH, midi_file))
    #print(score.show('text'))
    # Count the number of parts (tracks)
    num_tracks = len(score.parts)

    instruments = []
    for part in score.parts:
        inst = part.getElementsByClass('Instrument')
        #inst = part.getInstrument(returnDefault=True)
        instruments.append(str(inst[0]) if inst else "Unknown")
    
    instruments = score.recurse().getElementsByClass(instrument.Instrument)

    unique_instruments = set(instr.instrumentName or instr.bestName() for instr in instruments)

    # Extract time signatures
    time_signatures = [
        ts.ratioString for ts in score.recurse().getElementsByClass(meter.TimeSignature)
    ]
    num_time_signatures = len(set(time_signatures))


    # Extract tempo
    # tempos = [
    #     t.number for t in score.recurse().getElementsByClass(tempo.MetronomeMark)
    # ]
    
    # Extract key signature
    #key_signature = score.analyze('key')
    
    
    # Return metadata as a dictionary
    return pd.Series({
        "num_tracks": num_tracks,
        "num_time_signatures": num_time_signatures,
        "num_instruments": len(unique_instruments),
        "instruments": list(unique_instruments)
    })

def extract_midi_metadata_to_df(df):
    """
    Apply the extract_midi_metadata function to a dataframe and create new columns for metadata.

    Args:
        df (pd.DataFrame): The input dataframe.
    Returns:
        pd.DataFrame: The dataframe with new metadata columns.
    """

    # Apply the extract_midi_metadata function to each MIDI file in the dataframe
    #metadata_df = df['location'].apply(lambda file: pd.Series(extract_midi_metadata(os.path.join(DATA_PATH, file))))
    #metadata_df = df['location'].apply(extract_midi_metadata)

    metadata_df = DataFrameParallel(df, n_cores=-1, pbar=True)['location'].apply(extract_midi_metadata)

    # print(type(metadata_df))
    # Concatenate the original dataframe with the metadata dataframe
    return pd.concat([df, metadata_df], axis=1)

In [10]:
df_test = extract_midi_metadata_to_df(df2)

Process-2:   0%|          | 0/5984 [00:00<?, ?it/s]









Process-5:   0%|          | 0/5983 [00:00<?, ?it/s]



Process-0:   0%|          | 2/5984 [00:00<22:28,  4.44it/s]


Process-0:   0%|          | 3/5984 [00:00<22:22,  4.45it/s]


Process-0:   0%|          | 5/5984 [00:03<1:16:42,  1.30it/s]

Process-0:   0%|          | 8/5984 [00:03<34:27,  2.89it/s]  



Process-0:   0%|          | 9/5984 [00:03<31:22,  3.17it/s]

Process-0:   0%|          | 15/5984 [00:04<13:39,  7.29it/s]



Process-0:   0%|          | 17/5984 [00:06<41:13,  2.41it/s]




Process-0:   0%|          | 18/5984 [00:06<36:49,  2.70it/s]

















Process-0:   0%|          | 19/5984 [00:07<43:06,  2.31it/s]

Process-0:   0%|          | 20/5984 [00:07<36:20,  2.73it/s]















Process-0:   0%|          | 22/5984 [00:09<57:17,  1.73it/s]  


Process-0:   0%|          | 24/5984 [00:10<41:08,  2.41it/s]

Process-0:   0%|          | 25/5984 [00:10<40:07,  2.47it/s]





Process-0:   0%|          | 26/59

In [ ]:
#df_test.to_json('data/MidiCaps/train2.json', lines=True, orient='records')

In [13]:
df_test.head()

,location,caption,genre,genre_prob,mood,mood_prob,key,time_signature,tempo,tempo_word,...,chord_summary_occurence,instrument_summary,instrument_numbers_sorted,all_chords,all_chords_timestamps,test_set,num_tracks,num_time_signatures,num_instruments,instruments
2,lmd_full/1/16b9a230fb007c0009feee532c3c4686.mid,This energetic electronic and classical compos...,"[electronic, classical]","[0.21480000000000002, 0.1806]","[energetic, film, melodic, happy, dark]","[0.10790000000000001, 0.1062, 0.09580000000000...",C major,4/4,120.0,Moderate tempo,...,2,[],[],"[C, G, C, C7, Dm, C, G, C, C7, Bbmaj7, C, G7, ...","[0.464399092, 4.086712018, 7.430385487, 8.9164...",False,1,1,1,[None]
3,lmd_full/1/17655598958db48a34cd882f81402568.mid,A short electronic ambient song featuring a pi...,"[electronic, ambient]","[0.30310000000000004, 0.2369]","[film, energetic, melodic, epic, dark]","[0.11620000000000001, 0.11080000000000001, 0.1...",E major,4/4,170.0,Presto,...,5,[Piano],[0],"[B, Emaj7, C#m7, B7, A, B, Emaj7, C#m7, B7, A,...","[0.464399092, 1.30031746, 2.972154195, 3.71519...",False,1,1,1,[Piano]
6,lmd_full/1/180b3a492c1c1a46d005e762d62b9aa4.mid,A cheerful pop song with a touch of electronic...,"[pop, electronic]","[0.4238, 0.2548]","[happy, love, melodic, christmas, motivational]","[0.1521, 0.09870000000000001, 0.0935, 0.0737, ...",D major,4/4,96.0,Moderate tempo,...,4,"[Piano, Drums]","[0, 0, 0, 128, 0]","[D, A7, D, Em, A, D, Em7, A7, D, Em7, A7, D, G...","[0.464399092, 7.616145124, 9.938140589, 12.353...",False,5,1,5,"[bass, Piano, guitar, Percussion, strings]"
12,lmd_full/1/1f94f64f72af98fd92f206293b281f6f.mid,A classical piece that could be part of a film...,"[classical, soundtrack]","[0.6144000000000001, 0.225]","[film, relaxing, emotional, documentary, drama]","[0.16640000000000002, 0.1048, 0.0655, 0.0621, ...",C# major,4/4,74.0,Andante,...,2,[Recorder],[74],"[C#, Fm, Ab, C#, Fm, Bbm, C, Bbm, B, Bb, Ab, F...","[0.464399092, 1.764716553, 9.752380952, 11.609...",False,2,1,1,[Recorder]
17,lmd_full/1/112139e8b17be9cae1bd14df56e52f14.mid,A short fragment of energetic electronic and c...,"[electronic, classical]","[0.32370000000000004, 0.30610000000000004]","[energetic, film, dark, epic, action]","[0.1081, 0.1009, 0.0961, 0.0916, 0.0896]",F major,4/4,136.0,Allegro,...,1,[Piano],"[0, 0]","[A, F/A]","[0.464399092, 1.671836734]",False,2,1,2,"[intro bass (MIDI), Piano]"


In [ ]:
df_test = extract_midi_metadata_to_df(df.head(10))

In [ ]:
df3 = extract_midi_metadata_to_df(df2)

In [22]:
df3.to_json('data/MidiCaps/train_metadata.json', lines=True, orient='records')

In [21]:
df3.head()

,location,caption,genre,genre_prob,mood,mood_prob,key,time_signature,tempo,tempo_word,...,instrument_summary,instrument_numbers_sorted,all_chords,all_chords_timestamps,test_set,num_tracks,num_time_signatures,num_instruments,instruments,title
6,lmd_full/1/180b3a492c1c1a46d005e762d62b9aa4.mid,A cheerful pop song with a touch of electronic...,"[pop, electronic]","[0.4238, 0.2548]","[happy, love, melodic, christmas, motivational]","[0.1521, 0.09870000000000001, 0.0935, 0.0737, ...",D major,4/4,96.0,Moderate tempo,...,"[Piano, Drums]","[0, 0, 0, 128, 0]","[D, A7, D, Em, A, D, Em7, A7, D, Em7, A7, D, G...","[0.464399092, 7.616145124, 9.938140589, 12.353...",False,5,1,5,"[guitar, strings, Percussion, Piano, bass]",None
97,lmd_full/1/1c13264442d86027af49d6ebc8756b61.mid,A melodic electronic and pop song featuring a ...,"[electronic, pop]","[0.27990000000000004, 0.254]","[melodic, happy, love, christmas, motivational]","[0.1378, 0.0729, 0.0644, 0.061700000000000005,...",E minor,4/4,148.0,Vivace,...,"[Piano, Drums]","[0, 0, 0, 0, 128, 0, 0, 0, 0, 0, 0, 0, 0, 0]","[Em, C, Em, Am, C, Em, Am7, C, Em, C6, Em, Cma...","[0.464399092, 9.659501133, 13.003174603, 16.25...",False,1,1,13,"[StringInstrument, Electric Guitar, Electric P...",None
143,lmd_full/1/121a91bf6564c378e800137371459318.mid,A soothing classical piece featuring an orches...,"[classical, easylistening]","[0.4469, 0.1555]","[relaxing, meditative, christmas, film, melodic]","[0.16140000000000002, 0.11620000000000001, 0.1...",F minor,4/4,96.0,Moderato,...,"[Orchestral Harp, English Horn]","[46, 46, 69]","[C#, Cmaj7, Fm, Cm, Gm, D, G, C, Bm, F#m, Gm7b...","[0.464399092, 1.486077097, 2.507755102, 3.7151...",False,3,3,2,"[English Horn, Harp]",None
167,lmd_full/1/1d0c250a2a5f7aef30634ae830160127.mid,A short classical piece with electronic elemen...,"[classical, electronic]","[0.36660000000000004, 0.2197]","[film, melodic, happy, dark, energetic]","[0.12560000000000002, 0.1077, 0.1047, 0.0954, ...",Bb major,2/4,130.0,Fast,...,"[Piano, Harpsichord]","[0, 6]","[F7/A, Bb]","[0.464399092, 1.857596371]",False,1,1,6,"[Harpsichord, lyrics, Conductor Track, Les Pet...",None
203,lmd_full/1/12eafbd1632f9975db71a0a43defabbb.mid,An electronic soundtrack composition in E mino...,"[electronic, soundtrack]","[0.3553, 0.1542]","[dark, energetic, happy, film, melodic]","[0.1038, 0.08420000000000001, 0.0824, 0.0795, ...",E minor,4/4,96.0,Moderate tempo,...,"[Distortion Guitar, Synth Voice]","[30, 54, 30, 30]","[A7, F#7, Em, E, Em, Em7b5, A7, G, E, D, E, D,...","[1.021678004, 2.972154195, 4.922630385, 11.052...",False,4,2,5,"[Guitar 3 Lead, Electric Guitar, Voice, Guitar...",None


In [ ]:


def count_initial_silence_notes(path, silence_threshold_beats=8):
    """
    Count how many notes/rests are silent at the very beginning of a MIDI file.
    
    Args:
        midi_path (str): path to the midi file
        silence_threshold_beats (float): stop counting after this many beats of silence

    Returns:
        int: number of silent/rest notes at the beginning
    """
    file_path = os.path.join(DATA_PATH, path)
    score = converter.parse(file_path)
    flat = score.flatten().notesAndRests.stream()

    silent_count = 0
    accumulated_beats = 0.0

    for el in flat:
        if isinstance(el, note.Rest):
            silent_count += 1
            accumulated_beats += el.quarterLength
        elif isinstance(el, note.Note) and el.volume.velocity == 0:
            silent_count += 1
            accumulated_beats += el.quarterLength
        elif isinstance(el, chord.Chord) and all(p.volume.velocity == 0 for p in el.notes):
            silent_count += 1
            accumulated_beats += el.quarterLength
        else:
            # First sounding note/chord appears
            break

        if accumulated_beats > silence_threshold_beats:
            break

    return silent_count


def count_slient_notes_to_df(df):
    """
    Apply the extract_midi_metadata function to a dataframe and create new columns for metadata.

    Args:
        df (pd.DataFrame): The input dataframe.
    Returns:
        pd.DataFrame: The dataframe with new metadata columns.
    """

    # Apply the extract_midi_metadata function to each MIDI file in the dataframe
    #metadata_df = df['location'].apply(lambda file: pd.Series(extract_midi_metadata(os.path.join(DATA_PATH, file))))
    #n_silence = df['location'].apply(count_initial_silence_notes)

    n_silence = SeriesParallel(df['location'], n_cores=-1, pbar=True).apply(count_initial_silence_notes)

    n_silence_series = pd.Series(n_silence, name="initial_silence")
    print(type(n_silence))
    # print(type(metadata_df))
    # Concatenate the original dataframe with the metadata dataframe
    return pd.concat([df, n_silence_series], axis=1)

In [ ]:
df3 = df3.dropna()

In [ ]:
df4 = count_slient_notes_to_df(df3)

Incorrect files:
* lmd_full/1/17fecee2cf25f4d3d2eecb3e670a2df9.mid
* lmd_full/1/14bdef979c3d2e9816dff833d9a4be29.mid

In [55]:
def count_total_unique_instruments(midi_path):
    score = converter.parse(midi_path)
    all_instrs = score.recurse().getElementsByClass(instrument.Instrument)
    unique_instr_names = set(instr.instrumentName or instr.bestName() for instr in all_instrs)
    return  len(all_instrs), len(unique_instr_names), unique_instr_names

In [12]:
midi_file = 'data/MidiCaps/lmd_full/1/10c0d2e1ee5c21ec98b94d237bbef7e7.mid'
metadata = extract_midi_metadata(midi_file)
# ins_counts, unique_ins, ins_name = count_total_unique_instruments(midi_file)
# print(ins_counts, unique_ins)
# print(ins_name)
print(metadata)

{'num_tracks': 9, 'num_time_signatures': 1, 'num_instruments': 8, 'instruments': ['P1-Grand Piano', 'Acoustic Bass', 'Electric Piano', 'P2-Electric Piano 1', 'Brass', 'Percussion', 'Trumpet', 'Sampler'], 'title': None}


In [4]:
def read_midi(filename):
    """Read a MIDI file and return its contents."""
    with open(filename, 'rb') as f:
        return f.read()
        # Process the MIDI data as needed

In [5]:
filename = 'data/MidiCaps/lmd_full/1/1a0751ad20e2f82957410a7510a1b13e.mid'
read_midi(filename)

b"MThd\x00\x00\x00\x06\x00\x01\x00\t\x00xMTrk\x00\x00\x005\x00\xffX\x04\x04\x02\x18\x08\x00\xffY\x02\x00\x00\x00\xffQ\x03\x06\xc8\x1c\x00\xff\x06\x18Goldrake, colonna sonora\x00\xff/\x00MTrk\x00\x00\x03\xe2\x00\xff!\x01\x00\x00\xff\x03\x08Stringhe\x00\xc00\x00\xb0\x07\x7f\x00\xb0\n2\x00\xb0\x00\x00\x00\x0b\x7f\x00[x\x00]2\x83`\x90Ed\x00B<\nQPOB\x00\x00E\x00\nQ\x00\x15Dd\x00@<\nPPO@\x00\x00D\x00\nP\x00\x15=<\x00Bd\nNP\x13B\x00\x00=\x00\x01@<\x00Dd\tN\x00\x01PP'D\x00\x00@\x00\nP\x00\x01B<\x00Ed\nQP\x81\x01E\x00\x00B\x00\nQ\x00\x01EF\x00Id\nUPYI\x00\x00E\x00\nU\x00\x0bDF\x00Gd\nSPYG\x00\x00D\x00\nS\x00\x0bBF\x00Ed\nQP\x13E\x00\x00B\x00\x01DF\x00Gd\tQ\x00\x01SP'G\x00\x00D\x00\x01EF\x00Id\tS\x00\x01UP\x81\x01U\x00\x00I\x00\x00E\x00\x15XP\x00Id\x00LdwL\x00\x00I\x00\x00X\x00\x01MP\x00UP\x00IP\x85\x13I\x00\x00M\x00<U\x00=I\x7f;I\x00\x01N\x7f\x81\x01N\x003I\x7f;I\x00\x01Q\x7f\x81\x01Q\x00oS\x7f;S\x00\x01Q\x7f;Q\x00\x01P\x7f;P\x00\x01N\x7f;N\x00\x01P\x7fwP\x00\x01I\x7f;I\x00\x01I\x7f;I\x00\x01M\